In [28]:
# !pip install llama-index-llms-cleanlab llama-index llama-index-embeddings-huggingface

Setup LLM

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

from llama_index.llms.cleanlab import CleanlabTLM


options = {
    "model": "gpt-4o",
    "max_tokens": 256,
    "log": ["explanation"],
}

llm = CleanlabTLM(api_key=os.environ["CLEANLAB_API_KEY"], options=options)

In [18]:
response = llm.complete("What is NVIDIA's ticker symbol?")
print(response)

NVIDIA's ticker symbol is NVDA.


In [19]:
response.additional_kwargs

{'trustworthiness_score': 0.9885545474223644,
 'explanation': 'Did not find a reason to doubt trustworthiness.'}

In [20]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from utils import (
    setup_trustworthiness_handler,
    display_response
)


Settings.llm = llm

In [4]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [21]:
from typing import Dict, List, ClassVar
from llama_index.core.instrumentation.events import BaseEvent
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation import get_dispatcher
from llama_index.core.instrumentation.events.llm import LLMCompletionEndEvent


class GetTrustworthinessScoreAndReasoning(BaseEventHandler):
    events: ClassVar[List[BaseEvent]] = []
    trustworthiness_score: float = 0.0
    reasoning: str = ""

    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "GetTrustworthinessScoreAndReasoning"

    def handle(self, event: BaseEvent) -> Dict:
        if isinstance(event, LLMCompletionEndEvent):
            self.trustworthiness_score = event.response.additional_kwargs[
                "trustworthiness_score"
            ]
            self.reasoning = event.response.additional_kwargs[
                "explanation"
            ]
            self.events.append(event)


# Root dispatcher
root_dispatcher = get_dispatcher()

# Register event handler
event_handler = GetTrustworthinessScoreAndReasoning()
root_dispatcher.add_event_handler(event_handler)

def display_response(response):
    response_str = response.response
    trustworthiness_score = event_handler.trustworthiness_score
    reasoning = event_handler.reasoning
    print(f"Response: {response_str}")
    print(f"Trustworthiness score: {round(trustworthiness_score, 2)}")
    print(f"Reasoning: {reasoning}")


In [22]:
# Optional: Define `display_response` helper function


# This method presents formatted responses from our TLM-based RAG pipeline. It parses the output to display both the text response itself and the corresponding trustworthiness score.


In [23]:
# LlamaParse PDF reader for PDF Parsing
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data(
    "/Users/akshay/Eigen/ai-engineering-hub/trustworthy-rag/docs/dspy.pdf"
)
# Started parsing the file under job_id b76a572b-d2bb-42ae-bad9-b9810049f1af

Started parsing the file under job_id 3e4a97ad-7b24-45e2-af2f-45c549e10e88


In [24]:
index = VectorStoreIndex.from_documents(documents)

In [25]:
query_engine = index.as_query_engine()

In [26]:
response = query_engine.query("What is is DSPy?")
print(response)

DSPy is a framework that abstracts the process of prompting and fine-tuning language models by using natural language signatures. These signatures are typed declarations of functions that specify what a text transformation needs to do, rather than how to prompt a specific language model to achieve that behavior. DSPy signatures are used to create modules, such as the Predict module, which can be instantiated to perform tasks like question-answering. DSPy also includes more sophisticated modules that generalize prompting techniques, allowing for the creation of modular functions that support any signature. The framework aims to improve the quality of simple programs by automating and optimizing the prompting process without the need for hand-crafted prompts.


In [27]:
display_response(response)

Response: DSPy is a framework that abstracts the process of prompting and fine-tuning language models by using natural language signatures. These signatures are typed declarations of functions that specify what a text transformation needs to do, rather than how to prompt a specific language model to achieve that behavior. DSPy signatures are used to create modules, such as the Predict module, which can be instantiated to perform tasks like question-answering. DSPy also includes more sophisticated modules that generalize prompting techniques, allowing for the creation of modular functions that support any signature. The framework aims to improve the quality of simple programs by automating and optimizing the prompting process without the need for hand-crafted prompts.
Trustworthiness score: 0.96
Reasoning: Did not find a reason to doubt trustworthiness.
